In [1]:
# Step 1: Imports
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from PIL import Image

In [2]:
# Step 2: Paths
CSV_PATH = '../data/captions.csv'
IMG_FOLDER = '../data/train2017'
FEATURE_SAVE_PATH = '../features/image_features.npy'
IMAGE_SIZE = (299, 299)


In [ ]:
# Step 3: Load the CSV
df = pd.read_csv(CSV_PATH)
image_paths = df['image'].unique()  # Only need unique images

print(f"Found {len(image_paths)} unique images.")

In [3]:
# Step 4: Load InceptionV3 (remove top classification layer)
base_model = InceptionV3(weights='imagenet')
cnn_model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)


In [ ]:
# Step 5: Image Preprocessing Function
def load_and_preprocess_image(img_path):
    img_path = os.path.join(IMG_FOLDER, os.path.basename(img_path))
    img = keras_image.load_img(img_path, target_size=IMAGE_SIZE)
    img_array = keras_image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

In [ ]:
# Step 6: Extract Features
features = {}
for img_name in tqdm(image_paths):
    try:
        img_array = load_and_preprocess_image(img_name)
        feature_vector = cnn_model.predict(img_array)
        features[img_name] = feature_vector.squeeze()  # Shape: (2048,)
    except Exception as e:
        print(f"⚠️ Error processing {img_name}: {e}")

In [ ]:

# Step 7: Save features as numpy array
# Map image_name to row index, and save separately
image_list = list(features.keys())
feature_matrix = np.array([features[img] for img in image_list])

np.save(FEATURE_SAVE_PATH, feature_matrix)
np.save('../features/image_names.npy', image_list)

print(f"✅ Extracted features for {len(image_list)} images.")
print(f"Saved to: {FEATURE_SAVE_PATH}")